In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint


### Extract CSVs into DataFrames

In [3]:

covid_df = "covid.csv"
covid_df = pd.read_csv(covid_df, encoding='Latin-1')
covid_df.head(50)

covid_df.columns= covid_df.columns.str.lower()
for col in covid_df.columns:
    print(col)
    
covid_df.head()


entry_id
state patient number
date announced
age bracket
gender
detected city
detected district
detected state
state code
num cases
current status
contracted from which patient (suspected)
notes
source_1
source_2
source_3
nationality
type of transmission
status change date
patient number


/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (2,4,5,6,7,8,10,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,entry_id,state patient number,date announced,age bracket,gender,detected city,detected district,detected state,state code,num cases,current status,contracted from which patient (suspected),notes,source_1,source_2,source_3,nationality,type of transmission,status change date,patient number
0,311688.0,NaN,1/10/2020,NaN,NaN,NaN,Aizawl,Mizoram,MZ,16.0,Hospitalized,NaN,NaN,http://twitter.com/dipr_mizoram/status/1311478...,NaN,NaN,NaN,NaN,NaN,NaN
1,311689.0,NaN,1/10/2020,NaN,NaN,NaN,Champhai,Mizoram,MZ,0.0,Hospitalized,NaN,NaN,http://twitter.com/dipr_mizoram/status/1311478...,NaN,NaN,NaN,NaN,NaN,NaN
2,311690.0,NaN,1/10/2020,NaN,NaN,NaN,Kolasib,Mizoram,MZ,10.0,Hospitalized,NaN,NaN,http://twitter.com/dipr_mizoram/status/1311478...,NaN,NaN,NaN,NaN,NaN,NaN
3,311691.0,NaN,1/10/2020,NaN,NaN,NaN,Lawngtlai,Mizoram,MZ,5.0,Hospitalized,NaN,NaN,http://twitter.com/dipr_mizoram/status/1311478...,NaN,NaN,NaN,NaN,NaN,NaN
4,311692.0,NaN,1/10/2020,NaN,NaN,NaN,Champhai,Mizoram,MZ,-1.0,Recovered,NaN,NaN,http://twitter.com/dipr_mizoram/status/1311478...,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#Transforming the data to Postgres 
rds_connection_string = "postgres:postgres@localhost:5432/Covid_db"
engine = create_engine(f'postgresql://{rds_connection_string}')
engine.table_names()

/var/folders/w0/dyll3qjj14n0d4qd0jv9g_4w0000gn/T/ipykernel_56276/737504648.py:4: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


[]

In [5]:
#connect table to postgres
covid_df.to_sql(name='covid_info', con=engine, if_exists='append', index=False)
engine.table_names()

/var/folders/w0/dyll3qjj14n0d4qd0jv9g_4w0000gn/T/ipykernel_56276/4067823277.py:3: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['covid_info']

In [6]:
#testing the table after connecting to ensure connection worked
pd.read_sql_query('select * from covid_info', con=engine).head()

,entry_id,state patient number,date announced,age bracket,gender,detected city,detected district,detected state,state code,num cases,current status,contracted from which patient (suspected),notes,source_1,source_2,source_3,nationality,type of transmission,status change date,patient number
0,311688.0,None,1/10/2020,NaN,None,None,Aizawl,Mizoram,MZ,16.0,Hospitalized,None,None,http://twitter.com/dipr_mizoram/status/1311478...,None,None,None,None,None,None
1,311689.0,None,1/10/2020,NaN,None,None,Champhai,Mizoram,MZ,0.0,Hospitalized,None,None,http://twitter.com/dipr_mizoram/status/1311478...,None,None,None,None,None,None
2,311690.0,None,1/10/2020,NaN,None,None,Kolasib,Mizoram,MZ,10.0,Hospitalized,None,None,http://twitter.com/dipr_mizoram/status/1311478...,None,None,None,None,None,None
3,311691.0,None,1/10/2020,NaN,None,None,Lawngtlai,Mizoram,MZ,5.0,Hospitalized,None,None,http://twitter.com/dipr_mizoram/status/1311478...,None,None,None,None,None,None
4,311692.0,None,1/10/2020,NaN,None,None,Champhai,Mizoram,MZ,-1.0,Recovered,None,None,http://twitter.com/dipr_mizoram/status/1311478...,None,None,None,None,None,None


In [8]:
#connect table to postgres
covid_df.to_sql(name='covid_clean1', con=engine, if_exists='append', index=False)
engine.table_names()

/var/folders/w0/dyll3qjj14n0d4qd0jv9g_4w0000gn/T/ipykernel_56276/2028321314.py:3: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['covid_info', 'covid_info1', 'covid_clean', 'covid_clean1']

In [9]:
#connect table to postgres
sales_df.to_sql(name='covid_clean1', con=engine, if_exists='append', index=False)
engine.table_names()

,entry_id,state patient number,date announced,age bracket,gender,detected city,detected district,detected state,state code,num cases,current status,contracted from which patient (suspected),notes,source_1,source_2,source_3,nationality,type of transmission,status change date,patient number
0,311688.0,None,1/10/2020,NaN,None,None,Aizawl,Mizoram,MZ,16.0,Hospitalized,None,None,http://twitter.com/dipr_mizoram/status/1311478...,None,None,None,None,None,None
1,311689.0,None,1/10/2020,NaN,None,None,Champhai,Mizoram,MZ,0.0,Hospitalized,None,None,http://twitter.com/dipr_mizoram/status/1311478...,None,None,None,None,None,None
2,311690.0,None,1/10/2020,NaN,None,None,Kolasib,Mizoram,MZ,10.0,Hospitalized,None,None,http://twitter.com/dipr_mizoram/status/1311478...,None,None,None,None,None,None
3,311691.0,None,1/10/2020,NaN,None,None,Lawngtlai,Mizoram,MZ,5.0,Hospitalized,None,None,http://twitter.com/dipr_mizoram/status/1311478...,None,None,None,None,None,None
4,311692.0,None,1/10/2020,NaN,None,None,Champhai,Mizoram,MZ,-1.0,Recovered,None,None,http://twitter.com/dipr_mizoram/status/1311478...,None,None,None,None,None,None
